Clickhouse documentation chat

Iterate all .md files in the directory recursively: /Users/brainco/BrainCo/ML/clickhouse-docs

In [30]:
import os
import fnmatch

dir_path = '/Users/brainco/BrainCo/ML/clickhouse/docs' # replace with the actual path to the directory containing .md files
md_files = []
for subdir, dirs, files in os.walk(dir_path):
    for file in files:
        if fnmatch.fnmatch(file, '*.md'):
            filepath = os.path.join(subdir, file)
            md_files.append(filepath)

print(f'Number of .md files: {len(md_files)}')

Number of .md files: 1717


In [31]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import MarkdownTextSplitter


splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=150)

# Load the .md files into a list of Document objects
# All docs
all_docs = []
for md_file in md_files:
    loader = UnstructuredMarkdownLoader(md_file)
    data = loader.load()
    docs = splitter.split_documents(data)
    # Append every doc in docs to all_docs
    for doc in docs:
        all_docs.append(doc)

print(f'Number of documents: {len(all_docs)}')


Number of documents: 11246


In [ ]:
pip3 install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os

# initialize pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],  # find at app.pinecone.io
    environment="us-west1-gcp-free"  # next to api key in console
)

index_name = "clickhouse-docs"
embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(all_docs, embeddings, index_name=index_name)


# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

# query = "arrayDifference()"
# docs = docsearch.similarity_search(query)

/Users/brainco/BrainCo/ML/langchain-tutorials/venv/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


NameError: name 'all_docs' is not defined

In [33]:
query = "array join usage"
docs = docsearch.similarity_search(query)
print(docs)

[Document(page_content="The arrayJoin function affects all sections of the query, including the WHERE section. Notice the result 2, even though the subquery returned 1 row.\n\nExample:\n\nsql\nSELECT sum(1) AS impressions\nFROM\n(\n    SELECT ['Istanbul', 'Berlin', 'Bobruisk'] AS cities\n)\nWHERE arrayJoin(cities) IN ['Istanbul', 'Berlin'];\n\ntext\n┌─impressions─┐\n│           2 │\n└─────────────┘\n\nA query can use multiple arrayJoin functions. In this case, the transformation is performed multiple times and the rows are multiplied.\n\nExample:\n\nsql\nSELECT\n    sum(1) AS impressions,\n    arrayJoin(cities) AS city,\n    arrayJoin(browsers) AS browser\nFROM\n(\n    SELECT\n        ['Istanbul', 'Berlin', 'Bobruisk'] AS cities,\n        ['Firefox', 'Chrome', 'Chrome'] AS browsers\n)\nGROUP BY\n    2,\n    3", metadata={'source': '/Users/brainco/BrainCo/ML/clickhouse/docs/en/sql-reference/functions/array-join.md'}), Document(page_content='slug: /en/sql-reference/statements/select/arra

In [6]:
from langchain.chains.llm import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate
import pinecone
import os

# initialize pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],  # find at app.pinecone.io
    environment="us-west1-gcp-free"  # next to api key in console
)

# Replace with your own OpenAI API key
os.environ['OPENAI_API_KEY']='sk-xxxxxxxx'

index_name = "clickhouse-docs"
docsearch = Pinecone.from_existing_index(embedding=OpenAIEmbeddings(), index_name=index_name)

# llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0.7)

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
When your response contains code, format them into a markdown code block.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

# llm = OpenAI()

question_generator = LLMChain(llm=llm, prompt=PromptTemplate.from_template(_template))

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

qa_with_source = ConversationalRetrievalChain(
    retriever=docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain
)

chat_history = []
query3 = "How to use retention(), please give an code example, also please write a query using retention() to calculate user retention"
result = qa_with_source({'question': query3, 'chat_history': chat_history})
print(result)


ValueError: OpenAIChat currently only supports single prompt, got ["Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n2. Group users by unique ID uid using the retention function.\n\nQuery:\n\nsql\nSELECT\n    uid,\n    retention(date = '2020-01-01', date = '2020-01-02', date = '2020-01-03') AS r\nFROM retention_test\nWHERE date IN ('2020-01-01', '2020-01-02', '2020-01-03')\nGROUP BY uid\nORDER BY uid ASC\n\nResult:\n\ntext\n┌─uid─┬─r───────┐\n│   0 │ [1,1,1] │\n│   1 │ [1,1,1] │\n│   2 │ [1,1,1] │\n│   3 │ [1,1,1] │\n│   4 │ [1,1,1] │\n│   5 │ [0,0,0] │\n│   6 │ [0,0,0] │\n│   7 │ [0,0,0] │\n│   8 │ [0,0,0] │\n│   9 │ [0,0,0] │\n│  10 │ [0,0,0] │\n│  11 │ [0,0,0] │\n│  12 │ [0,0,0] │\n│  13 │ [0,0,0] │\n│  14 │ [0,0,0] │\n└─────┴─────────┘\n\n3. Calculate the total number of site visits per day.\n\nQuery:\n\nsql\nSELECT\n    sum(r[1]) AS r1,\n    sum(r[2]) AS r2,\n    sum(r[3]) AS r3\nFROM\n

In [35]:
import json
json.dump(result, open('result.json', 'w'), indent=4)

In [42]:
query3 = "How to use retention(), please give a code example"
result = qa_with_source({'question': query3, 'chat_history': chat_history})
print(result)

PineconeProtocolError: Failed to connect; did you specify the correct index name?